In [1]:
import os
cwd = os.getcwd()
print(cwd)
if cwd == '/home/jovyan':
    os.chdir('/home/jovyan/work/lunaricorn/notebooks')
!pip install --upgrade pip
!pip install -r ../lunaricorn/requirements.txt

/home/jovyan/work/lunaricorn/notebooks


In [1]:
import sys
import os
from pathlib import Path
from datetime import datetime
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(Path(os.getcwd()).parent.absolute())
import lunaricorn
import lunaricorn.api.signaling as lsig

# Redirect all logging output to Jupyter cell output
import logging

# Create a handler that writes log messages to stdout (Jupyter cell output)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

# Set a simple log format
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(name)s: %(message)s')
handler.setFormatter(formatter)

# Get the root logger and configure it
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)
# Remove all existing handlers
for h in root_logger.handlers[:]:
    root_logger.removeHandler(h)
root_logger.addHandler(handler)

# Also set logging for common noisy libraries to WARNING
for noisy in ['asyncio', 'urllib3', 'selenium', 'playwright', 'websockets']:
    logging.getLogger(noisy).setLevel(logging.WARNING)


In [9]:
sig_cfg = lsig.SignalingClientConfig("192.168.0.18", 5555, 5556, 5557)
sig_client_id = "API_TEST_NOTEBOOK"

sig_client = lsig.SignalingClient(sig_cfg, sig_client_id)
rc = sig_client.connect()
if not rc:
    print("cannot connect")

2025-10-12 14:30:01,064 [INFO] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Connected to REP socket at tcp://192.168.0.18:5555
2025-10-12 14:30:01,065 [INFO] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Connected to PUB socket at tcp://192.168.0.18:5555
2025-10-12 14:30:01,066 [INFO] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: ZeroMQ client API_TEST_NOTEBOOK connected successfully
2025-10-12 14:30:01,066 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:30:11,068 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:30:21,070 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:30:31,072 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:30:41,073 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK

In [12]:
# push test
import uuid
import random
import string

def make_url() -> str:
    random_string = ''.join(random.choices(string.ascii_lowercase, k=10))
    name_based_uuid = uuid.uuid5(uuid.NAMESPACE_URL, random_string)
    return f"https://{random_string}.example.com?u={name_based_uuid}"
    
agent_id = ''.join(random.choices(string.ascii_lowercase, k=120))
sig_client.push_event(event_type = lsig.SignalingEventType.Robots.value,
                            payload = {"job_enter": "rss_loader", "url": make_url()},
                            source=agent_id,
                            tags=[lsig.SignalingEventFlags.JobEntry.value]
                            )
for i in range(100):
    dt = datetime.now().strftime("%Y%m%d%H%M%S%f")[:-3]
    filename = ''.join(random.choices(string.ascii_lowercase, k=120))
    sig_client.push_event(event_type = lsig.SignalingEventType.Robots.value,
                               payload = {"title": "e.title"*30, "filename": f"{i}_{filename}", "dt": dt},
                               source=agent_id,
                               tags=[lsig.SignalingEventFlags.News.value, lsig.SignalingEventFlags.Md.value, lsig.SignalingEventFlags.Obsidian.value]
                               )
# for end   
sig_client.push_event(event_type = lsig.SignalingEventType.Robots.value,
                                            payload = {"job_exit": "rss_loader", "url": make_url()},
                                            source=agent_id,
                                            tags=[lsig.SignalingEventFlags.JobExit.value]
                                            )

{'status': 'success', 'eid': 472}

2025-10-12 14:31:50,957 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:31:50,961 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK


In [13]:
sig_client.get_tags_list()

['JobExit', 'News', 'Obsidian', 'Md', 'JobEntry']

2025-10-12 14:32:00,959 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:00,963 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK


In [14]:
sig_client.get_types_list()

['push', 'robots']

In [6]:
sig_client.get_affected_list()

['form_data_67890', 'user_session_12345', 'database_primary', 'storage_server']

In [15]:
sig_client.get_owners_list()

['ownerless']

2025-10-12 14:32:10,961 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:10,964 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK


In [16]:
sig_client.browse_events(0.0, event_types = sig_client.get_types_list())

[ClientEventData(eid=472, event_type='robots', payload={'job_exit': 'rss_loader', 'url': 'https://zowrglgjaa.example.com?u=1c3997e0-a3cf-53ca-b913-a79d63008781'}, timestamp=1760279510.906024, source='ownerless', affected=None, tags=['JobExit']),
 ClientEventData(eid=471, event_type='robots', payload={'dt': '20251012143150874', 'filename': '99_mkhjnvmjueqbcfzljgeusrpuumqclzpyclxawjjpznsqmytlwxrtkecaooeivjffggyhziaolkjswccnhffyfeaemzmwejqlubazzynvbtqpyfnooguaxmoj', 'title': 'e.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.titlee.title'}, timestamp=1760279510.874196, source='ownerless', affected=None, tags=['News', 'Md', 'Obsidian']),
 ClientEventData(eid=470, event_type='robots', payload={'dt': '20251012143150842', 'filename': '98_vocqenetsbjyusutglokyodtdvbodptrqcpyvnfsdtnfpxdidydobqlhbjnlqhyolpwglfpasvcsqnityqopvliqpiwnmdcnhyzllhticdcdzdy

2025-10-12 14:32:20,962 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:20,965 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:30,963 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:30,967 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:40,965 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:40,968 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:50,966 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
2025-10-12 14:32:50,969 [DEBUG] lunaricorn.api.signaling.client.cid#API_TEST_NOTEBOOK: Heartbeat sent from API_TEST_NOTEBOOK
